In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
tf.__version__

'2.10.0'

In [3]:
import os

In [4]:
rootdir = './data/'
labels = []

for file in os.listdir(rootdir):
    labels.append(file)
    

In [5]:
labels

['03', '1', '100519733001', '100519733006', '100519733015', '2', '7']

In [6]:
x, y = [] , []


In [7]:
from PIL import Image

In [8]:
for index , label in enumerate(labels):
    path = rootdir+ label
    for file in os.listdir(path):
        image = Image.open(os.path.join( path, file) , "r")
        image = np.asarray(image)
        x.append(image / 255)
        y.append(index)

In [9]:
len(x)

1134

In [10]:
x = np.asarray(x)
y = np.asarray(y)

In [11]:

len(y)

1134

In [12]:
type(x[6])

numpy.ndarray

In [13]:
x[0]

array([[0.84705882, 0.84313725, 0.84313725, ..., 0.74901961, 0.74901961,
        0.74901961],
       [0.84313725, 0.84313725, 0.84313725, ..., 0.74901961, 0.74901961,
        0.74901961],
       [0.84313725, 0.84313725, 0.84313725, ..., 0.75294118, 0.74901961,
        0.74901961],
       ...,
       [0.23921569, 0.23921569, 0.23529412, ..., 0.87058824, 0.8745098 ,
        0.87843137],
       [0.23137255, 0.23137255, 0.23137255, ..., 0.8745098 , 0.8745098 ,
        0.87843137],
       [0.21960784, 0.22352941, 0.22352941, ..., 0.87843137, 0.87843137,
        0.87843137]])

In [14]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[256, 256]),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(150, activation="relu"),
    tf.keras.layers.Dense(len(labels), activation="softmax")
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 65536)             0         
                                                                 
 dense (Dense)               (None, 200)               13107400  
                                                                 
 dense_1 (Dense)             (None, 150)               30150     
                                                                 
 dense_2 (Dense)             (None, 7)                 1057      
                                                                 
Total params: 13,138,607
Trainable params: 13,138,607
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.layers

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
xtrain , xtest , ytrain , ytest = train_test_split(x , y, test_size = 0.2)

In [19]:
len(xtrain)

907

In [20]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam" , metrics = ["accuracy"])

In [21]:
xtrain , xvalid = xtrain[:140], xtrain[140:]
ytrain , yvalid = ytrain[:140] , ytrain[140:]

In [22]:
len(xtrain)

140

In [23]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('model1ANN.h5' , save_best_only = True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=7,restore_best_weights=True)


In [24]:
xtrain = np.asarray(xtrain)

In [25]:
xtrain.shape

(140, 256, 256)

In [26]:
history = model.fit(xtrain ,ytrain , epochs = 30,validation_data = (xvalid, yvalid) ,
                    callbacks = [checkpoint_cb , early_stopping_cb])

Epoch 1/30
5/5 [==============================] - 4s 597ms/step - loss: 23.7597 - accuracy: 0.1214 - val_loss: 9.9620 - val_accuracy: 0.3416
Epoch 2/30
5/5 [==============================] - 1s 239ms/step - loss: 8.0431 - accuracy: 0.4000 - val_loss: 10.6267 - val_accuracy: 0.5385
Epoch 3/30
5/5 [==============================] - 2s 414ms/step - loss: 5.8358 - accuracy: 0.7000 - val_loss: 1.9442 - val_accuracy: 0.6858
Epoch 4/30
5/5 [==============================] - 2s 361ms/step - loss: 1.2642 - accuracy: 0.7643 - val_loss: 0.6876 - val_accuracy: 0.8605
Epoch 5/30
5/5 [==============================] - 1s 348ms/step - loss: 0.3980 - accuracy: 0.9143 - val_loss: 0.4936 - val_accuracy: 0.9074
Epoch 6/30
5/5 [==============================] - 1s 230ms/step - loss: 0.2628 - accuracy: 0.9286 - val_loss: 0.5539 - val_accuracy: 0.9074
Epoch 7/30
5/5 [==============================] - 1s 340ms/step - loss: 0.1125 - accuracy: 0.9500 - val_loss: 0.3528 - val_accuracy: 0.9205
Epoch 8/30
5/5 [==

In [27]:
preds = model.predict(xtest)

8/8 [==============================] - 0s 18ms/step


In [28]:
x = []
for pred in preds:
    x.append(np.argmax(pred))

In [29]:
from sklearn.metrics import confusion_matrix

In [30]:
confusion_matrix(ytest, x)

array([[26,  0,  0,  1,  0,  0,  0],
       [ 0, 40,  0,  1,  0,  0,  0],
       [ 2,  0, 37,  0,  0,  0,  0],
       [ 0,  0,  1, 36,  0,  0,  0],
       [ 0,  0,  0,  0, 42,  0,  0],
       [ 0,  0,  0,  0,  0,  5,  0],
       [ 0,  0,  1,  0,  0,  0, 35]], dtype=int64)